# Graph Neural Networks

---
**Lernziele**
* Sie verstehen, wie Moleküle für den Computer als Graph dargestellt werden können. 
* Sie verstehen, wie Graph Neural Networks funktionieren.
* Sie können ein Graph Neural Network als Pytorch Klasse schreiben.
---

Graph Neural Networks sind noch eine relativ neue Methode. Intuitiv können wir sehen, dass sich Moleküle sehr gut als (mathematischen) Graph darstellen lassen. Dabei entsprechen die Bindungen im Molekül den Kanten (edges) des Graph und die Atome  den Knoten (nodes).<br>Für den Computer sind jedoch Graphen nicht so leicht zu lesen wie zum Beispiel ein SMILES string. Wir stellen die Moleküle als zwei Matrizen dar. Zum einen die Adjacency-Matrix, die die Atomverknüpfung (also die Bindungen) darstellt. Dazu kommen dann noch die Feature Matrix. Für das Machine Learning One-Hot encoden wir hier die Atome.<br><br>
<center>
<img src="https://www.researchgate.net/profile/Jorge_Galvez2/publication/236018587/figure/fig1/AS:299800013623305@1448489301609/The-chemical-graph-and-adjacency-matrix-of-the-isopentane.png" style="width: 600px;">
</center>
<h8><center>Galvez et. al. 2010</center></h8><br><br>
Wir nehmen nochmal die Daten aus der Tox21 Challenge. 

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils import data
import math
from sklearn.metrics import roc_auc_score
%run ../utils/onehotencoder.py
np.set_printoptions(linewidth=300)

In [2]:
# Laden der Daten
data_tox = pd.read_csv("../data/toxicity/sr-mmp.tab", sep = "\t")
data_tox = data_tox.iloc[:,1:] #alle Spalten bis auf die erste (index 0) werden ausgewählt
data_tox.columns = ["smiles", "activity"]
data_tox.head()

,smiles,activity
0,OC(=O)[C@H](O)[C@@H](O)[C@H](O)C(=O)CO,0
1,C[C@]12CC[C@H]3[C@@H](CCc4cc(O)ccc43)[C@@H]1CC...,1
2,CC(C)(C)c1cc(O)ccc1O,1
3,CN(C)c1ccc(cc1)C(c1ccccc1)=C1C=CC(C=C1)=[N+](C)C,1
4,NC(Cc1ccccc1)C(O)=O,0


## Adjacency Matrix und One-Hot Encoded Feature Matrix
Mit den Smiles können wir jetzt leider noch nicht so viel anfangen. Aber zum Glück gibt es in RDKit direkt eine Funktion, um die Adjacency Matrix zu erstellen. Diese haben wir oben schon importiert.

In [3]:
mols = [Chem.MolFromSmiles(x) for x in data_tox['smiles']]
A = [GetAdjacencyMatrix(x) for x in mols]
print(A[1])

[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 

Um die Atome zu One-Hot Encoden benutzen wir die vorgeschriebene `onehotencode()` Funktion.

In [4]:
feat = onehotencode(mols)
feat[1]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Vielleicht ist ihnen aufgefallen, dass auf der Diagonalen der Adjacency Matrix noch Nullen stehen. In einer Graph Convolution sollen aber nicht nur die Features der benachbarten, sondern auch des zentralen Atoms mit in die Rechnung genommen werden. Dafür werden Einsen auf der Diagonalen der Adjacency-Matrix benötigt. 
Die Funktion `np.fill_diagonal(matrix, value)` erlaubt es Ihnen die Werte der Diagonalen einer Matrix zu ändern. 

In [5]:
for matrix in A:
    np.fill_diagonal(matrix, 1)
print(A[1])

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 

## Graph Convolution
Jetzt wollen wir die Informationen der Knoten an die Nachbarknoten entlang der Kanten weiterzugeben, was als (Graph) Convolution bezeichnet wird. Hierfür werden während des forward pass diese Mathematischen Operationen des ausgeführ: $$\hat{X} = \hat{D}^{-1}\hat{A}XW$$
Hier ist $\hat{A}$ unsere Adjacency Matrix mit Einsen auf der Diagonalen. $X$ ist die Feature-Matrix und $W$ sind die Weights, die pytorch initialisiert. Es fehlt noch $D$, die Degree Matrix. Diese Matrix enthält die Anzahl der Verbindungen, die jedes Atom im Molekül hat. Diese Werte werden in der Degree-Matrix auf der Diagonalen platziert. Die Anzahl der Verbindung jedes Moleküls lässt sich leicht aus der Adjacency-Matrix berechnen. Dafür müssen Sie die Summe der Reihe oder Spalten der$\hat{A}$ Matrix berechnen. Diese Summe gibt die Degrees eines Atom an. Um genau zu sein, ist es der Degree + 1 des Atoms, da Sie die Diagonale der Adjacency-Matrix bereits mit Einsen gefüllt haben.
Um die Degree-Matrix zu erstellen, müssen Sie die Summen der Reihen in jeder Adjacency-Matrix berechnen und dieses auf die Diagonale einer neuen Matrix platzieren.


Dieser Prozess ist relativ simpel mit `numpy`. `np.sum(A[i], axis=0)` berechnet die Summe pro Spalte und `np.diag()` erstellt aus einem 1D-Array eine Matrix mit den Werten des 1D Arrays auf der Diagonalen. Sie können die beiden Funktion in Kombination benutzen, um die Degree-Matrizen zu erstellen.

In [6]:
D =[]
for matrix in A:
    D.append(np.diag(np.sum(matrix, axis=1)))
print(D[1])

[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 

Bevor wir anfangen ein Netzwerk zu bauen und es mit unseren Daten zu füttern, können wir noch einen Schritt durchführen um einiges an Rechenaufwand zu sparen, denn $\hat{D}^{-1}\hat{A}$ kann schon vor dem Trainieren des Netzwerk berechnet werden. Dadurch kann Zeit gespart werden, da dieser Schritt nicht ständig im Netzwerk wiederholt werden muss.

`np.linalg.matrix_power` wird benötigt, um das Invers der Matrix `D` zu berechnen. Ansonsten ist dieser Schritt eine einfache Matrixmultiplikation.

In [9]:
DA = []
for i in range(len(D)):
    DA.append(np.matmul(np.linalg.matrix_power(D[i], -1),A[i]))

In [10]:
DA = [torch.tensor(x,dtype=torch.float32) for x in DA] # Konvertieren der Arrays zu Tensoren
feat = [torch.tensor(x,dtype=torch.float32) for x in feat] # Konvertieren der Arrays zu Tensoren
labels = [torch.tensor([x], dtype=torch.float32) for x in data_tox['activity']]

## Graph Convolution Layer
Wir wollen uns für die Graph Convolution die Sturktur von PyTorch zunutze machen. Wie schon letzte Woche verwenden wir dafür Klassen. Unser Ziel ist es, so etwas wie nn.Linear() zu programmieren, sodass wir unser Repertoire an Hidden Layers (Linear, RNN, GRU, Dropout, ...) um eine Graph Convolution Layer erweitern.<br>Auch dafür können wir die nn.Module Klasse als Basis für unsere GraphConvolution verwenden. Wir fangen mit dem minimalen Grundgerüst an:
```python
class GraphConvolution(nn.Module):
    pass
```
Damit wir richtig auf die Funktionen der übergeordneten nn.Module Klasse zugreifen können müssen wir diese mit `super().__init__()` initialisieren. Unsere Convolutional Layer muss natürlich wissen, wie groß der input ist, und wie groß der output werden soll. Außerdem müssen wir hier selbst die Weights und Biases initialiseren.
```python
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_featuers = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
```
Dann können wir auch schon die forward() Funktion programmieren. Hier passiert die eigentliche convolution, bei der die Knoten mit den Features `x` durch Matrixmultiplikation mit der Adjacency Matrix `adj` über ihre Nachbarknoten lernen. Indem wir noch die lernbaren `weights` dazwischenschalten, können wir das ganze später optimieren.
```python
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
```
Damit würde unsere GraphConvolution Klasse auch schon funktionieren. Wir initialisieren die weights aber noch etwas optimiert mit der Funktion `reset_parameters()`. Und mit der Funktion `__repr__()` können wir uns die Layer noch richtig anzeigen lassen. Unsere finale Klasse sieht dann so aus:

In [11]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()
    
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(0, stdv)
        self.bias.data.uniform_(-stdv, stdv)
        
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + 'in_features=' + str(self.in_features) + ', ' \
               + 'out_features=' + str(self.out_features) + ')'

Wir können eine einzelne Graph Convolution Layer auch schon verwenden:

In [12]:
feat_beispiel = feat[1]
adj_beispiel = DA[1]
print('Features:', feat_beispiel.shape)
print('DA:', adj_beispiel.shape)

conv = GraphConvolution(feat[1].shape[1], 100)
output = conv(feat_beispiel, adj_beispiel)
print('\nOutput:', output.size())

Features: torch.Size([29, 25])
DA: torch.Size([29, 29])

Output: torch.Size([29, 100])


## Graph Neural Network
Um jetzt ein Netz zu bauen, können wir wieder die nn.Module Basisklasse verwenden. 

# Hier
# noch 
# Text 
# und
# Erklärungen
hardcoded hidden layer shapes?

In [13]:
class GraphNN(nn.Module):
    def __init__(self):#in_features, out_features, size_labels):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        # self.lin2 (and add relu if forward)
        
    def aggregate(self, convoluted_graph): # we use mean aggregation, max or min could also be used as hyperparameter
        return torch.mean(convoluted_graph, dim=0)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x

## Training und Evaluation

# Testset
# Fehlt

In [19]:
gnn = GraphNN()
loss_funktion= nn.BCEWithLogitsLoss()
optimizer=optim.Adam(gnn.parameters(), lr =0.001)

In [15]:
print(len(feat))
print(len(DA))
print(len(labels))

2246
2246
2246


In [20]:
EPOCHS = 50

for i in range(EPOCHS):
    loss_list = []
    acc_list= []
    for k in range(len(feat)):
        optimizer.zero_grad()
    
        output=gnn(feat[k], DA[k])

        loss=loss_funktion(output,labels[k])
        loss.backward()
        loss_list.append(loss.item())
        optimizer.step()
    
    # Evaluierung nach dem EPOCH an Hand des Testsets
    #acc_list= []
    #for k in range(len(feat)):
        #output=gnn(feat[k], DA[k])#
        acc_list.append(np.sum((torch.round(torch.sigmoid(output)) == labels[k]).detach().numpy()))
        
        
    print(i,"Test Loss: %.2f Test Accuracy: %.2f"
        % (np.mean(loss_list), np.sum(acc_list)/(len(feat))))

0 Test Loss: 0.67 Test Accuracy: 0.58
1 Test Loss: 0.64 Test Accuracy: 0.63
2 Test Loss: 0.63 Test Accuracy: 0.64
3 Test Loss: 0.62 Test Accuracy: 0.66
4 Test Loss: 0.62 Test Accuracy: 0.67
5 Test Loss: 0.61 Test Accuracy: 0.68
6 Test Loss: 0.61 Test Accuracy: 0.68
7 Test Loss: 0.61 Test Accuracy: 0.68
8 Test Loss: 0.60 Test Accuracy: 0.68
9 Test Loss: 0.60 Test Accuracy: 0.68
10 Test Loss: 0.60 Test Accuracy: 0.68
11 Test Loss: 0.60 Test Accuracy: 0.68
12 Test Loss: 0.60 Test Accuracy: 0.69
13 Test Loss: 0.60 Test Accuracy: 0.69
14 Test Loss: 0.60 Test Accuracy: 0.69
15 Test Loss: 0.60 Test Accuracy: 0.69
16 Test Loss: 0.59 Test Accuracy: 0.69
17 Test Loss: 0.59 Test Accuracy: 0.69
18 Test Loss: 0.59 Test Accuracy: 0.70
19 Test Loss: 0.59 Test Accuracy: 0.70
20 Test Loss: 0.59 Test Accuracy: 0.70
21 Test Loss: 0.59 Test Accuracy: 0.70
22 Test Loss: 0.59 Test Accuracy: 0.70
23 Test Loss: 0.59 Test Accuracy: 0.70
24 Test Loss: 0.58 Test Accuracy: 0.70
25 Test Loss: 0.58 Test Accuracy: 0

PyG hat schon Convolutional, Pooling etc Layers und Batching

## Übungsaufgabe: ?